## Tensorflow 파일에서 데이터 읽어와서 학습

Why? 엄청난 양의 training set의 데이터를 학습할려면 소스코드 입력시 너무 복잡하다
파일이 굉장히 커서 메모리에 올리기 힘들다 
Queue Runner System 사용
여러개의 파일을 큐에다가 쌓은후 Reader로 연결한 후 데이터 양식에 맡게 Decoder 한 후 다시 큐에 write 
내가 필요한 batch 만큼 데이터를 가져와 learning 

#### Queue Runner 의 3가지 단계
![1](https://user-images.githubusercontent.com/33486820/48301109-8dca4d00-e52b-11e8-91d8-f4fc9b02b8c6.jpg)
1. 파일들의 리스트를 만들어 준다.
- 파일 네임을 리스트 한다

2. 파일을 읽어 올 Reader 를 정해준다.
- 텍스트 파일을 읽을 때 사용

3. Value를 어떻게 parsing 할 것인지
- decode_csv를 가지고 분석 한다 읽어 올때 각각 필드 값을 정의 , Default도 마찬가지


In [ ]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
# batch를 가지고 읽어 온다 읽어 온 데이터를 x,y, 데이터를 batch한다
#batch_size 는 한번에 몇개를 가지고 올 것인가 정의
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
#Queue 를 관리하기 위해 사용 하는 함수 ==> Tensorflow 가 관리
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))
